In [1]:
import gym

In [2]:
import numpy as np
import operator
from collections import defaultdict


class QLearningAgent():
    def __init__(self, possibleActions, learningRate, discountFactor, epsilon, initVals=0.0):
        self.possibleActions = possibleActions
        self.learningRate = learningRate
        self.discountFactor = discountFactor
        self.epsilon = epsilon
        self.state = []
        self.G = initVals
        self.prevState = []
        self.prevAction = None
        self.prevReward = initVals
        self.currState = []
        self.currAction = None
        self.QValues = defaultdict(lambda: {})
        self.policy = defaultdict(float)
        self.numPossActions = len(self.possibleActions)

    def getPolicy(self):
        # Generate e-greedy policy
        if self.state in self.QValues.keys():
            for action in self.possibleActions:
                self.policy[action] = self.epsilon / self.numPossActions
            bestAction = max(self.QValues[self.state].items(), key=operator.itemgetter(1))[0]
            self.policy[bestAction] += (1.0 - self.epsilon)
        else:
            for action in self.possibleActions:
                self.policy[action] = 1 / self.numPossActions

        #print("Policy to take action : ")
        #for k, v in self.policy.items():
        #   print("action ",k,"Choose prob ",v)

    def toStateRepresentation(self, state):
        return tuple((state[0], state[1]))

    def setExperience(self, state, action, reward):
        # Save data from t-1 - First step in episode
        if self.prevState == []:
            self.prevState = state
            self.prevAction = action
            self.prevReward = reward

        self.currState = state
        self.currAction = action
        self.currReward = reward

    def setState(self, state):
        self.state = state

    def reset(self):
        self.state = []
        self.prevState = []
        self.prevAction = None
        self.prevReward = 0.0
        self.currState = []
        self.currAction = None
        self.currReward = 0.0

    def act(self):
        # Take an action
        self.getPolicy()
        probs = list(self.policy.values())
        actions = list(self.policy.keys())
        action = actions[np.random.choice(np.arange(len(probs)), p=probs)]
        return action

    def setEpsilon(self, epsilon):
        self.epsilon = epsilon

    def computeHyperparameters(self):
        return self.epsilon

    def learn(self):

        print("-----Learning update-----")

        #Initialize Q[state][action] to zero
        if not self.currState in self.QValues.keys():
            for action in self.possibleActions:
                self.QValues[self.currState][action] = 0

        if not self.prevState in self.QValues.keys():
            for action in self.possibleActions:
                self.QValues[self.prevState][action] = 0

        # If action is none, step is terminal and Q[state][action] is zero.
        if self.currAction != None:
            target = self.prevReward + (self.discountFactor * self.QValues[self.currState][self.currAction])
        else:
            target = self.prevReward

        delta = target - self.QValues[self.prevState][self.prevAction]
        self.QValues[self.prevState][self.prevAction] += self.learningRate * delta

        # Save the current values for next time
        self.prevState = self.currState
        self.prevAction = self.currAction
        self.prevReward = self.currReward

        return self.QValues







In [3]:

if __name__ == "__main__":

    env = gym.make("Blackjack-v0")
    space_size = env.action_space.n
    possibleActions = []
    for i in range(space_size):
        possibleActions.append(str(i))

    # Initialize a SARSA Agent
    agent = QLearningAgent(possibleActions, learningRate=0.1, discountFactor=0.999, epsilon=0.9)

    # Run training SARSA Method
    for episode in range(30):
        print("\n******************************* EPISODE ", episode, "***************************")
        agent.reset()
        observation = env.reset()  # Returns current state
        nextObservation = None
        epsStart = True
        print("State after reset: ", observation)

        done = False

        while done == False:

            epsilon = agent.computeHyperparameters()
            agent.setEpsilon(epsilon)
            obsCopy = observation  # Copy current state
            agent.setState(agent.toStateRepresentation(obsCopy))
            action = agent.act()

            print("Action: ", action)

            nextObservation, reward, done, status = env.step(int(action))

            best_next_action = np.argmax(agent.QValues[nextObservation])

            print("Next state: ", nextObservation)
            print("Next reward: ", reward)

            print("Episode finished: ", done)
            agent.setExperience(agent.toStateRepresentation(obsCopy), str(best_next_action), reward)

            if not epsStart:
                agent.learn()
            else:
                epsStart = False

            observation = nextObservation

        agent.setExperience(agent.toStateRepresentation(nextObservation), None, None)
        agent.learn()

        QValues = agent.QValues



******************************* EPISODE  0 ***************************
State after reset:  (16, 2, False)
Action:  0
Next state:  (16, 2, False)
Next reward:  1.0
Episode finished:  True
-----Learning update-----

******************************* EPISODE  1 ***************************
State after reset:  (21, 5, True)
Action:  1
Next state:  (18, 5, False)
Next reward:  0.0
Episode finished:  False
Action:  0
Next state:  (18, 5, False)
Next reward:  0.0
Episode finished:  True
-----Learning update-----
-----Learning update-----

******************************* EPISODE  2 ***************************
State after reset:  (7, 6, False)
Action:  1
Next state:  (17, 6, False)
Next reward:  0.0
Episode finished:  False
Action:  0
Next state:  (17, 6, False)
Next reward:  1.0
Episode finished:  True
-----Learning update-----
-----Learning update-----

******************************* EPISODE  3 ***************************
State after reset:  (7, 1, False)
Action:  1
Next state:  (15, 1, False)

In [4]:

if __name__ == "__main__":

    env = gym.make("Blackjack-v0")
    space_size = env.action_space.n
    possibleActions = []
    for i in range(space_size):
        possibleActions.append(str(i))

    # Initialize a SARSA Agent
    agent = QLearningAgent(possibleActions, learningRate=0.1, discountFactor=0.999, epsilon=0.9)

    # Run training SARSA Method
    for episode in range(50000):
        agent.reset()
        observation = env.reset()  # Returns current state
        nextObservation = None
        epsStart = True

        done = False

        while done == False:

            epsilon = agent.computeHyperparameters()
            agent.setEpsilon(epsilon)
            obsCopy = observation  # Copy current state
            agent.setState(agent.toStateRepresentation(obsCopy))
            action = agent.act()

            nextObservation, reward, done, status = env.step(int(action))

            best_next_action = np.argmax(agent.QValues[nextObservation])

            agent.setExperience(agent.toStateRepresentation(obsCopy), str(best_next_action), reward)

            if not epsStart:
                agent.learn()
            else:
                epsStart = False

            observation = nextObservation

        agent.setExperience(agent.toStateRepresentation(nextObservation), None, None)

        agent.learn()

        QValues = agent.QValues


-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

...

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

In [5]:
print("Q value function: ")

print(QValues)

# m = 0
# for k, v in sorted(QValues.items()):
#     m +=1
#     print("item ",m,k,v)
#
# for k, v in sorted(QValues.items()):
#     print("state: ",k," action: ",max(v.items(), key=operator.itemgetter(1))[0], " value: ",max(v.items(), key=operator.itemgetter(1))[1])



Q value function: 
defaultdict(<function QLearningAgent.__init__.<locals>.<lambda> at 0x7f5c2c5be9d0>, {(14, 6, False): {}, (14, 6): {'0': -0.23669216006778057, '1': 0}, (18, 2, False): {}, (18, 2): {'0': -0.3392874567807806, '1': 0}, (17, 3, False): {}, (17, 3): {'0': -0.5167374827077006, '1': 0}, (13, 1, False): {}, (23, 1, False): {}, (13, 1): {'0': -0.8932233353283537, '1': 0}, (12, 1): {'0': -0.8606580627671419, '1': 0}, (23, 1): {'0': 0, '1': 0}, (12, 8, False): {}, (12, 8): {'0': -0.6833716926510034, '1': 0}, (10, 7, False): {}, (10, 7): {'0': -0.19361387731812263, '1': 0}, (18, 10, False): {}, (18, 10): {'0': -0.6126751331955921, '1': 0}, (14, 10): {'0': -0.6730936002041266, '1': 0}, (30, 10, False): {}, (30, 10): {'0': 0, '1': 0}, (20, 10): {'0': -0.426312242007447, '1': 0}, (12, 9, False): {}, (19, 9, False): {}, (12, 9): {'0': -0.46437316694028463, '1': 0}, (14, 9): {'0': -0.6642876378240486, '1': 0}, (19, 9): {'0': -0.32008991569728645, '1': 0}, (21, 10, False): {}, (21, 10